In [1]:
import sys
sys.path.insert(0, '/Users/cliff/side_projects/python_functions')
from web_scraping import activate_chrome_selenium_latest

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from glob import glob
import pandas as pd
import numpy as np
import warnings
import pygsheets
import math
import time
import re
import os

from nlb_functions import get_book_urls_on_page

warnings.filterwarnings('ignore')

### Clean files first 

In [2]:
file_list = glob("*.rtf")
len(file_list)

0

In [3]:
for files in file_list:
    os.remove(files)

### Go to start the scraping 

In [4]:
browser = activate_chrome_selenium()
# browser = webdriver.Firefox()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/cliff.chew/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [5]:
info = pd.read_csv('nlb_credentials.csv')
account_name = info['values'][0]
password = info['values'][1]

login_page = 'https://cassamv2.nlb.gov.sg/cas/login'
browser.get(login_page)

# Actions on login page
username_tag = """//*[@id="username"]"""
element = browser.find_element_by_xpath(username_tag)
element.send_keys("{}".format(account_name))

password_tag = """//*[@id="password"]"""
element = browser.find_element_by_xpath(password_tag)
element.send_keys("{}".format(password))

login_button_2 = """//*[@id="fm1"]/section[4]/input[4]"""
browser.find_element_by_xpath(login_button_2).click()

### Add hit to get number of page iterations needed 

In [6]:
url_link = "https://www.nlb.gov.sg/mylibrary/Bookmarks"    
browser.get(url_link)
soup = bs(browser.page_source, 'html5lib')

In [7]:
max_records = float(soup.find_all("div", text=re.compile("Showing"))[0].text.split(" ")[-2])
range_list = range(1, int(math.ceil(max_records / 20)) + 1)
range_list

range(1, 5)

### Loop through the pages! 

In [8]:
book_urls_dict = dict()
for i in range_list:
    book_urls_list = list()
    url_link = "https://www.nlb.gov.sg/mylibrary/Bookmarks?q=&s=&a=&p={}".format(i)
    browser.get(url_link)
    soup = bs(browser.page_source, 'html5lib')
    book_urls_dict[i] = list(set(get_book_urls_on_page(soup, book_urls_list)))

In [9]:
all_book_url_lists = list()
for i in range(1, len(book_urls_dict) + 1):
    all_book_url_lists = all_book_url_lists + book_urls_dict[i]

In [10]:
final_table = pd.DataFrame()

In [ ]:
# Write iteration count
for i, urls in enumerate(all_book_url_lists):
    browser.get(urls)
    book = bs(browser.page_source, 'html5lib')
    time.sleep(3)

    try:
        link_on_book = """//*[@id="result-content-grid"]/div/div/div/div[2]/h5/a"""
        browser.find_element_by_xpath(link_on_book).click()
    except:
        pass

    link_on_availability = """//*[@id="mainContent"]/div[3]/div[2]/div[3]/div[1]/div/div/div[1]/a"""
    browser.find_element_by_xpath(link_on_availability).click()
    time.sleep(3)
    
    book = bs(browser.page_source, 'html5lib')
    with open("{}.rtf".format(urls.split('=')[-1]), "wb") as text_file:
        text_file.write(book.encode('utf-8'));

### Taking locally saved files and loading into Google 

In [ ]:
file_list = glob("*.rtf")
len(file_list)

In [ ]:
final_table = pd.DataFrame()
for files in file_list:
    with open(files, encoding="utf8") as f:
        book = bs(f.read())

    base_holder = list()
    base = list()
    title = book.find('title').get_text()
    try:
        for i in book.find(class_= 'table table-stacked'):
                for tr in i.find_all('tr'):
                    for td in tr.find_all('td'):
                        if "To locate" not in td.text:
                            base.append(td.text)

        for i in range(int(len(base) / 3)):
            base_holder.append(base[(i*3):((i+1)*3)])

        base_table = pd.DataFrame(base_holder)
        base_table['title'] = title
        final_table = final_table.append(base_table)
        # print(title)
    except:
        print(files)
        print(title + " is MISSNG!!")
        print("")

In [ ]:
len(final_table.title.drop_duplicates().tolist())

In [ ]:
final_table.columns = ['library', "number", "availability", 'title']
final_table = final_table[['library', 'title', 'number', 'availability']]
final_table.head()

### Past portion 

In [ ]:
final_table = final_table[~final_table['number'].isnull()]
final_table['number'] = [i.replace("English", "").replace("Chinese", "") for i in final_table['number']]
final_table = final_table[~final_table.availability.isnull()]

In [ ]:
final_table['availability'] = [i.replace("Onloan - Due: ", "") for i in final_table['availability']]
final_table['title'] = [i.split(r"/")[0].strip() for i in final_table['title']]
ffinal_table = final_table[(final_table.library=="Bishan Public Library")]
ffinal_table = ffinal_table.sort_values('availability')
ffinal_table.shape

### Cleaning Bookmarks Sheet 

In [ ]:
google_auth = "/Users/cliff.chew/Downloads/self/python_functions/API Project-8b57e830c88b.json"
gc = pygsheets.authorize(service_file=google_auth)
name_of_worksheet = 'NLB Project'
sh = gc.open(name_of_worksheet)

### Cleaning All Sheet 

In [ ]:
bishan = sh.worksheet_by_title("Bookmarks")
bishan.clear('A2:E1000')

In [ ]:
bishan_table = final_table[final_table.library.str.contains("Bishan")]
bishan.set_dataframe(bishan_table,(1,1))
bishan.update_value('F2', "=ARRAYFORMULA(vlookup(A2:A{}, Current_borrowed!$A$2:$D$16,2,False))".format(
    bishan_table.shape[0] + 1))

### All 

In [ ]:
all_ = sh.worksheet_by_title("All")
all_.clear('A2:F1000') 

In [ ]:
all_.set_dataframe(final_table,(1,1))
all_.update_value('F2', "=ARRAYFORMULA(vlookup(A2:A{}, Current_borrowed!$A$2:$D$16,2,False))".format(
    final_table.shape[0] + 1))

link to page: https://docs.google.com/spreadsheets/d/1s5oYU59jyU_QO3IIhCClyWGoC_MpW9L_h4l4djDUKO0/edit#gid=1021888748